In [1]:
import keras

Using TensorFlow backend.


In [7]:
import sklearn
from keras.models import model_from_json

In [8]:
import os

In [23]:
directory = "/Volumes/matlab_share/nifti_img/3DEGIR_SAG/merged"
SLICE_NUM = 185

In [9]:
json_string = open(os.path.join('/Users/sasayabaku/working/checkResult/10160855', 'result', '10160213', 'model.json')).read()

In [10]:
model = model_from_json(json_string)

In [12]:
model.load_weights(os.path.join('/Users/sasayabaku/working/checkResult/10160855', 'result', '10160213', 'weights.hdf5'))

In [13]:
list_dir = "/Volumes/matlab_share/csv/"

In [33]:
import pandas as pd
import nilearn.image as nilimg
import sys
import numpy as np
from keras.utils import np_utils

In [27]:
patients_list = pd.read_csv(os.path.join(list_dir, 'patient_list.csv'),
                                   header=None,
                                   index_col=None, names=['dir', 'pid', 'page', 'psex'],
                                   dtype={'pid': 'object'})

In [28]:
        properties_list = pd.read_csv(os.path.join(list_dir, 'property.csv'),
                                    header=0,
                                    index_col=None,
                                    dtype={'0002': 'object'})

In [29]:
def encode_label(y):
    processed = y - 1
    processed = np_utils.to_categorical(processed, num_classes=2)
    return processed

In [67]:
predict_list = np.empty((0, 2)).astype(np.float32)

In [63]:
for pd_index, pd_data in patients_list.iterrows():

    patient = pd_data.dir

    try:
        nifti_files = os.listdir(os.path.join(directory, patient))
    except FileNotFoundError:
        continue

    raw_file = [file for file in nifti_files if file.startswith('c12018') & file.endswith('.nii')][0]
    personal_nifti = nilimg.smooth_img(os.path.join(directory, patient, raw_file), fwhm='fast')
    personal_img = personal_nifti.get_data()
    reshape_img = np.reshape(personal_img, (1, personal_img.shape[0], personal_img.shape[1], personal_img.shape[2]))
    if reshape_img.shape[3] < SLICE_NUM:
        train_img = np.append(reshape_img, np.zeros((1, reshape_img.shape[1], reshape_img.shape[2], SLICE_NUM - reshape_img.shape[3])), axis=3)
    elif reshape_img.shape[3] == SLICE_NUM:
        train_img = reshape_img
    else:
        sys.stderr.write('Data Size Error: This Nifti file size is ' + str(reshape_img.shape))
        sys.exit()

    pid = patients_list[patients_list.dir == str(patient)].pid.values[0]

    try:
        personal_sex = properties_list[properties_list['0002'] == pid]['0007'].values[0]
    except IndexError:
        continue
    train_label = encode_label(personal_sex)

    predict_sex = model.predict_classes(train_img)
    data = np.array([[int(train_label[0]), int(predict_sex[0])]])
    predict_list = np.append(predict_list, data, axis=0)

In [48]:
from sklearn.metrics import confusion_matrix
confusion_matrix(predict_list[:,0], predict_list[:,1].round(0))

array([[287,  14],
       [ 16, 386]])

In [65]:
predict_list.shape

(1765, 2)